In [14]:
import numpy as np
import pandas as pd 
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
dataset = pd.read_csv("Housing.csv")
X = dataset.iloc[:,1:]
y = dataset.iloc[:,0].values
y = y.reshape(len(y),1)
encode_column = ['mainroad','guestroom','basement','hotwaterheating','airconditioning','prefarea','furnishingstatus']
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),encode_column)],remainder='passthrough')
X = ct.fit_transform(X)
scx = StandardScaler()
scy = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

model = SVR(kernel='rbf')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(r2_score(y_test, y_pred))
print(np.concatenate((y_test.reshape(len(y_test),1),y_pred.reshape(len(y_pred),1)),1))

-0.04118520528930247
[[ 4585000.          4378491.43891782]
 [ 6083000.          4378571.86651723]
 [ 4007500.          4378482.21900045]
 [ 6930000.          4378550.2450299 ]
 [ 2940000.          4378485.46036482]
 [ 6195000.          4378539.23296008]
 [ 3535000.          4378488.73349795]
 [ 2940000.          4378482.53316486]
 [ 3500000.          4378483.04054449]
 [ 7980000.          4378570.82084107]
 [ 6755000.          4378532.3435909 ]
 [ 3990000.          4378490.70761066]
 [ 3150000.          4378482.06275491]
 [ 3290000.          4378533.55077526]
 [ 4130000.          4378532.04028486]
 [ 2660000.          4378469.45229952]
 [ 4410000.          4378509.32784211]
 [ 3710000.          4378532.74702315]
 [ 3360000.          4378476.96991303]
 [ 4270000.          4378501.09676804]
 [ 5005000.          4378565.4455116 ]
 [ 5383000.          4378541.12470456]
 [ 6440000.          4378511.4270359 ]
 [ 1890000.          4378464.87261927]
 [ 6125000.          4378540.56119888]
 [ 5

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
